In [1]:
import pandas as pd
import numpy as np
import json

# TRATAMENTO DE TEXTOS
import re
import nltk
stopwords=nltk.corpus.stopwords.words('portuguese')


In [2]:
df = pd.read_json(r'reviews_merge.json')
df = df[['title','avaliacao']]
df['avaliacao'] = df['avaliacao'].map({0:'Negativo',1:'Neutro',2:'Positivo', np.nan:'Indefinido'}, na_action=None)
base = [tuple(x) for x in df.to_numpy()]


In [3]:
df.query('avaliacao == "Indefinido"')

,title,avaliacao


In [4]:
def fazstemmer(frases):
    stemmer = nltk.stem.RSLPStemmer()
    frasessstemming = []
    for (palavras, emocao) in frases:
        comstemming = [str(stemmer.stem(p))
                       for p in palavras.split() if p not in stopwords]
        frasessstemming.append((comstemming, emocao))
    return frasessstemming
 
 
frasescomstemming = fazstemmer(base)
#print(frasescomstemming)

In [5]:
def buscapalavras(frases):
    todaspalavras = []
    for (palavras, emocao) in frases:
        todaspalavras.extend(palavras)
    return todaspalavras
 
todaspalavras = buscapalavras(frasescomstemming)
#print(todaspalavras)

In [6]:
def buscafrequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras
frequencia = buscafrequencia(todaspalavras)


In [7]:
def busca_palavrasunicas(frequencia):
    freq = frequencia.keys()
    return freq
 
palavrasunicas = busca_palavrasunicas(frequencia)
#print(palavrasunicas)

In [8]:
df.head()

,title,avaliacao
0,Empresa dinâmica e agradável de trabalhar.,Positivo
1,Não tem perspectiva de de crescimento de carreia,Negativo
2,Foi uma boa experiencia,Positivo
3,"Empresa muito boa de se trabalhar, da oportuni...",Positivo
4,Excelente empresa pra trabalhar,Positivo


In [9]:
def extrai_palavras(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavrasunicas:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [10]:
basecompleta = nltk.classify.apply_features(extrai_palavras,frasescomstemming)

In [11]:
classificador = nltk.NaiveBayesClassifier.train(basecompleta) 

In [12]:
df = df[['title']]

In [13]:
df.head()

,title
0,Empresa dinâmica e agradável de trabalhar.
1,Não tem perspectiva de de crescimento de carreia
2,Foi uma boa experiencia
3,"Empresa muito boa de se trabalhar, da oportuni..."
4,Excelente empresa pra trabalhar


In [14]:
frases_Novas = df.values.tolist()
stemmer = nltk.stem.RSLPStemmer()
frase_resultado =[]
cont = 0
for i in frases_Novas:
    frase = frases_Novas[cont][0]
    frase_i = []
    for (palavras) in frase.split():
        comstem = [p for p in palavras.split()]
        frase_i.append(str(stemmer.stem(comstem[0])))
    frase_resultado.append(frase_i)
    cont = cont + 1
#print(frase_resultado)

In [34]:
resultado = []
for i in frase_resultado:
    nova_frase = extrai_palavras(i)
 
    distribuicao = classificador.prob_classify(nova_frase).max()
    resultado.append(distribuicao)
    #print('-----------------------')
    #for classe in distribuicao.samples():
    #    print("%s: %f" % (classe, distribuicao.prob(classe)))
response = {
                'total':len(resultado),
                'positivo': resultado.count('Positivo'),
                'negativo': resultado.count('Negativo'),
                'neutro': resultado.count('Neutro')
            }


response


{'total': 711, 'positivo': 533, 'negativo': 135, 'neutro': 43}

In [ ]:
resultado =[]
id = 0
for i in frase_resultado:
    nova_frase = extrai_palavras(i)
    distribuicao = classificador.prob_classify(nova_frase)
    for classe in distribuicao.samples():
        if float(distribuicao.prob(classe)) > 0.50000:
            resultado.append([classe, distribuicao.prob(classe), id])
    id = id + 1


In [ ]:
print(resultado)

[['Positivo', 0.9989254181608643, 0], ['Negativo', 0.9976132130682884, 1], ['Positivo', 0.9799301940069632, 2], ['Neutro', 0.9609950290801835, 3], ['Positivo', 0.9997473554797592, 4], ['Positivo', 0.9919821767969101, 5], ['Neutro', 0.8672240020523168, 6], ['Positivo', 0.8473777721059028, 7], ['Positivo', 0.6925260192584818, 8], ['Positivo', 0.992076971887783, 9], ['Positivo', 0.8410771813382873, 10], ['Positivo', 0.9983921441881475, 11], ['Positivo', 0.9741331129678017, 12], ['Positivo', 0.9991799426308129, 13], ['Positivo', 0.9945331126504883, 14], ['Negativo', 0.6739116927441621, 15], ['Positivo', 0.9760388516199693, 16], ['Positivo', 0.9845108356878308, 17], ['Positivo', 0.7248091946605039, 18], ['Positivo', 0.9188143014744171, 19], ['Positivo', 0.9982045613682802, 20], ['Negativo', 0.9995477388861906, 21], ['Positivo', 0.9078856409850816, 22], ['Positivo', 0.903680498922975, 23], ['Negativo', 0.9994767757725364, 24], ['Positivo', 0.9968474307975431, 25], ['Positivo', 0.999337006090

In [ ]:
print(len(resultado))

696


In [ ]:
print(len(erro))

1431


In [ ]:
print(resultado)

[['Positivo', 0.9989254181608643, 0], ['Negativo', 0.9976132130682884, 1], ['Positivo', 0.9799301940069632, 2], ['Neutro', 0.9609950290801835, 3], ['Positivo', 0.9997473554797592, 4], ['Positivo', 0.9919821767969101, 5], ['Neutro', 0.8672240020523168, 6], ['Positivo', 0.8473777721059028, 7], ['Positivo', 0.6925260192584818, 8], ['Positivo', 0.992076971887783, 9], ['Positivo', 0.8410771813382873, 10], ['Positivo', 0.9983921441881475, 11], ['Positivo', 0.9741331129678017, 12], ['Positivo', 0.9991799426308129, 13], ['Positivo', 0.9945331126504883, 14], ['Negativo', 0.6739116927441621, 15], ['Positivo', 0.9760388516199693, 16], ['Positivo', 0.9845108356878308, 17], ['Positivo', 0.7248091946605039, 18], ['Positivo', 0.9188143014744171, 19], ['Positivo', 0.9982045613682802, 20], ['Negativo', 0.9995477388861906, 21], ['Positivo', 0.9078856409850816, 22], ['Positivo', 0.903680498922975, 23], ['Negativo', 0.9994767757725364, 24], ['Positivo', 0.9968474307975431, 25], ['Positivo', 0.999337006090